In [ ]:
#mount google drive so that we can save data to files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#download APTOS 2019 Blindness Detection from Kaggle
#https://www.kaggle.com/competitions/aptos2019-blindness-detection/data

!kaggle competitions download -c aptos2019-blindness-detection

100% 9.50G/9.51G [01:56<00:00, 61.7MB/s]
100% 9.51G/9.51G [01:56<00:00, 87.6MB/s]


In [ ]:
!unzip aptos2019-blindness-detection.zip

Streaming output truncated to the last 5000 lines.
  inflating: test_images/512beaada8f0.png  
  inflating: test_images/51323e9d2070.png  
  inflating: test_images/515675001b9e.png  
  inflating: test_images/515877da2def.png  
  inflating: test_images/5168ddccea88.png  
  inflating: test_images/5172c056a687.png  
  inflating: test_images/5177461f5339.png  
  inflating: test_images/51910bcb980d.png  
  inflating: test_images/51a3e9b6a1c0.png  
  inflating: test_images/51bbd82feec8.png  
  inflating: test_images/51de5fb8eccd.png  
  inflating: test_images/51e88694c81f.png  
  inflating: test_images/51f0bf8f1544.png  
  inflating: test_images/51f57fb32840.png  
  inflating: test_images/51fa42dc1415.png  
  inflating: test_images/5276ab33b7b0.png  
  inflating: test_images/52a353760f72.png  
  inflating: test_images/531937254e73.png  
  inflating: test_images/5325b1c22c61.png  
  inflating: test_images/53959e0a750b.png  
  inflating: test_images/53be22c72713.png  
  inflating: test_images/

In [ ]:
import os
os.listdir("/content")

['.config',
 'train_images',
 'kaggle.json',
 'aptos2019-blindness-detection.zip',
 'sample_submission.csv',
 'test.csv',
 'drive',
 'train.csv',
 '.ipynb_checkpoints',
 'test_images',
 'sample_data']

In [ ]:
!pip install pandas -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 64.6 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 2.2.2 which is incompatible.


In [ ]:
import pandas as pd
import pandas.testing as tm
import numpy as np

import os
os.chdir('/content/drive/My Drive/')
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
import time

import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from sklearn.model_selection import train_test_split

In [ ]:
train_data = pd.read_csv("/content/train.csv")

test_data = pd.read_csv("/content/test.csv")

sample_submission = pd.read_csv("/content/sample_submission.csv")


In [ ]:
#variable initialization
split_size = 0.15
im_width = 512
im_height = 512
class_labels = {0: 'No DR[0]',1: 'Mild[1]', 2: 'Moderate[2]', 3: 'Severe[3]', 4: 'Proliferative DR[4]'}

In [ ]:
#split data into train and validation sets
def split(train_data, size, is_split = True):
    try:
        if is_split:

            diagnoses = train_data['diagnosis']
            identifiers = train_data['id_code']
            training_ids, validation_ids, training_diagnoses, validation_diagnoses = train_test_split(identifiers, diagnoses, stratify=diagnoses, shuffle=True, test_size=size)

            print("Training data: {} {}".format(training_ids.shape, training_diagnoses.shape))
            print("Validation data: {} {}".format(validation_ids.shape, validation_diagnoses.shape))

            return training_ids, training_diagnoses, validation_ids, validation_diagnoses
        else:
            return train_data['id_code'], train_data['diagnosis'], [], []
    except:
        print("File must be .csv format.")

In [ ]:
train_x, train_labels, validation_x, validation_labels = split(train_data, split_size)

Training data: (3112,) (3112,)
Validation data: (550,) (550,)


In [ ]:
pip install pandas==2.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 37.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 2.0.0 which is incompatible.


In [ ]:
#save train, validation, and test data to files
train_dataframe = pd.DataFrame(train_x, columns = ['id_code'])
train_dataframe['diagnosis'] = train_labels
train_dataframe.to_csv("/content/drive/My Drive/train_dataframe.csv", index = False)

validation_dataframe = pd.DataFrame(validation_x, columns = ['id_code'])
validation_dataframe['diagnosis'] = validation_labels
validation_dataframe.to_csv('/content/drive/My Drive/validation_dataframe.csv', index = False)

test = pd.read_csv('/content/test.csv')
test.to_csv('/content/drive/My Drive/test.csv', index = False)

In [ ]:
train = pd.read_csv("/content/drive/My Drive/train_dataframe.csv")
validation = pd.read_csv("/content/drive/My Drive/validation_dataframe.csv")
train_x = train['id_code']
train_labels = train['diagnosis']
validation_x = validation['id_code']
validation_labels = validation['diagnosis']

In [ ]:
#preprocessing the images for use in model training: https://www.kaggle.com/ratthachat/aptos-eye-preprocessing-in-diabetic-retinopathy
class Preprocess:
    def __init__(self, im_width, im_height, std_dev=8, tolerance=7):
        self.std_dev = std_dev
        self.tolerance = tolerance
        self.im_height = im_height
        self.im_width = im_width

    #crop dark regions from grayscale image based on threshold value
    def crop_dark_region_2D(self, im, crop = False):
        if crop:
          mask = im>self.tolerance
          return im[np.ix_(mask.any(1),mask.any(0))]
        else:
          return im

    #crop dark regions from RGB image based on threshold value
    def crop_dark_region_3D(self, im, crop = False):
      if crop:
        gray_im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
        mask = gray_im>self.tolerance

        check_shape = im[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark
            return im
        else:
            im1 = im[:,:,0][np.ix_(mask.any(1),mask.any(0))]  #R
            im2 = im[:,:,1][np.ix_(mask.any(1),mask.any(0))]  #G
            im3 = im[:,:,2][np.ix_(mask.any(1),mask.any(0))]  #B
            im = np.stack([im1,im2,im3],axis=-1)
        return im

    #apply circular mask drawn from the center of the image.  used to focus on central portion of the image
    def apply_circular_mask(self,im, draw = True):
      if draw:
        x = int(self.im_width/2)
        y = int(self.im_height/2)
        r = np.amin((x,y))     # radius

        new_im = np.zeros((self.im_height, self.im_width), np.uint8)
        cv2.circle(new_im, (x,y), int(r), 1, thickness=-1)

        #apply circular mask
        im = cv2.bitwise_and(im, im, mask=new_im)

      return im

    #reduce noise using Gaussian smoothing
    def blur(self, im, gaussian_blur = False):
        if gaussian_blur:
          im = cv2.addWeighted(im,4,cv2.GaussianBlur(im,(0,0),self.std_dev),-4,128)
        return im

    def preprocess(self, im, crop = True, gaussian_blur = True):
        #check if RGB or grayscale, then crop dark regions
        if im.ndim == 2:
            im = self.crop_dark_region_2D(im, crop)
        else:
            im = self.crop_dark_region_3D(im, crop)
        im = cv2.resize(im, (self.im_height, self.im_width))  #resize image
        im = self.apply_circular_mask(im)  #apply circular mask
        im = self.blur(im, gaussian_blur) #apply gaussian blur
        return im

In [ ]:
#convert images to vector for storage in .npy format
def im_to_vec(data, data_type):
    start_time = time.time()

    im_vec = np.empty((len(data),im_width, im_height), dtype = np.uint8)

    preprocessing_obj = Preprocess(im_width, im_height, std_dev)
    if data_type !='test':
        c = '/content/train_images/'
    else:
        c = '/content/test_images/'
    for iter,row in enumerate(tqdm(data)):
        im_path = c+data.iloc[iter]+'.png'
        im = cv2.imread(im_path)
        im = preprocessing_obj.preprocess(im)    #perform image preprocessing
        im_vec[iter,:,:,:] = im

    path = '/content/drive/My Drive/preprocessed_images'

    if not os.path.exists(path):
        os.makedirs(path)
    np.save(path+'/'+data_type+'.npy', im_vec)

In [ ]:
std_dev = 14
im_to_vec(train_x, "training")
im_to_vec(validation_x,"validation")

In [ ]:
test = pd.read_csv("/content/test.csv")
im_to_vec(test['id_code'], 'test')